# Discover: Finding alliteration in Lucretius

## Devise
Let's turn now to another kind of literary formalism, namely alliteration. As we did in the previous notebook, the key is to identify the specific letters of interest in the specific words of interest in the specific works of interest. And for that reason our work is already pretty far along for this experiment. Like with the clearchics, we are interested in the first letter of each word, not only at the beginning of lines but sequential through out our texts. Take this line from Lucretius:

> Viva videns vivo sepeliri viscera busto. Lucr. *DRN* 5.993

Read it out loud and you feel your lips push out each *v*. You can hear the repeated sound. Looking at the line, we see the repeated vees. There is poetry in these lines, expressed in repetition and specifically expressed in formal repetition. How do we capture this in code? Or, sticking with our section definition, how to we discover alliteration in Latin poetry.

## Plan

How should we go about formalizing alliteration? We again start with pseudocode...

**Pseudocode for counting alliteration in Lucretius**

- Load our library of Latin texts, keeping only the works of Lucretius
- Get bigrams for each line of Lucretius
- Iterate over all lines and check for alliterative bigrams, i.e. two consecutive words with the same first letter
- Aggregate/display the results

## Code

In [ ]:
# Preliminary imports
from natsort import natsorted
from pprint import pprint
from time import sleep

Some things never change. We need our corpus reader and we need out Lucretian texts.

In [ ]:
# PC 1: Load our library of Latin texts, keeping only the works of Lucretius

from cltkreaders.lat import LatinTesseraeCorpusReader
from latintools import preprocess

T = LatinTesseraeCorpusReader()

lucretius = [file for file in T.fileids() if file.startswith('lucretius')]

To simplify our work up front, let's begin with a narrow definition of alliteration. In this first version of our experiment, we will say that alliteration occurs when two consecutive words begin with the same letter. This will prove to be both underspecified (what about three or more consecutive words?) and overly specific (what about two words in a line that begin with the same letter but are not consecutive?). But it is often a good idea to get a handle on a simple version of a problem and gradually introduce more complexity. So, the simple version, starting from the line from Lucretius quoted above...

In [ ]:
lucretius_docrows = list(T.doc_rows(lucretius)) 
example = lucretius_docrows[4]['<lucr. 5.993>'] # remember that 4 is Book 5 because of zero-indexing
print(example)

In [ ]:
#PC 2: Get bigrams for each line of Lucretius

# bigrams

example_words = example.split()
example_bigrams = list(zip(example_words, example_words[1:]))
example_bigrams

Now we can say that for any tuple consisting of two words, if both words start with the same letter, we can identify that pair as an example of alliteration.

In [ ]:
for bigram in example_bigrams:
    if bigram[0][0] == bigram[1][0]:
        print(bigram)

This checks outs. Let's formalize this idea into a function...

In [ ]:
def is_alliteration(bigram):
    return bigram[0][0] == bigram[1][0]

Note that we can just check for equality of the first letter. If they match this operator returns `True`, if not it returns `False`. No need to explicitly return these values.

In [ ]:
for bigrams in example_bigrams:
    print(f'{bigrams} -> {is_alliteration(bigrams)}')

The theme of this section is Discover. So let's use our function to search over all of Lucretius and return a comprehensive list of alliterative pairs.

In [ ]:
def bigrams(line):
    words = line.split()
    return list(zip(words, words[1:]))

In [ ]:
# PC 3: Iterate over all lines and check for alliterative bigrams, i.e. two consecutive words with the same first letter

alliterative_pairs = []

for doc_rows in lucretius_docrows:
    for citation, line in doc_rows.items():
        if line: # fix underlying data—shouldn't have to check if line is empty
            line_bigrams = bigrams(preprocess(line))
            for bigram in line_bigrams:
                if is_alliteration(bigram):
                    alliterative_pairs.append((citation, bigram))

In [ ]:
alliterative_pairs[:5]

In [ ]:
print(f'There are {len(alliterative_pairs)} alliterative pairs in Lucretius.')

There are many different directions that we could take this experiment at this point and I will ask you to consider some of these directions in the Explore section below. For now, let's take the opportunity to do a bit of philological visualization. Already from the first five examples, we can see that the letter *p* may be a common alliterative letter. How does it compare to other letters? Munro, for example, suggests that Lucretius's "alliterations comprise almost every letter of the alphabet." Let's put this all in data-driven perspective.

In [ ]:
# PC 4: Aggregate/display the results

from collections import Counter
import re

latin_letters = "".join([word[0] for word in T.words(lucretius, preprocess=preprocess)])
latin_letters = set(re.sub(r'[^a-z]', '', latin_letters))

alliterative_letters = Counter()
alliterative_letters.update({k: 0 for k in list(latin_letters)})

for _, alliterative_pair in alliterative_pairs:
    alliterative_letters[alliterative_pair[0][0]] += 1

In [ ]:
import pandas as pd


df = pd.DataFrame(alliterative_letters.most_common(), columns=['letter', 'count']).sort_values('letter').reset_index(drop=True)
df

Munro is right—*almost* every letter of the alphabet. While there are words beginning with *z* in Lucretius, there are no alliterative *z* pairs.

In [ ]:
df.plot.bar(x='letter', y='count', rot=0, figsize=(10, 5));

So, we see now that *p* is a common alliterative letter, but not the *most* common. That honor belongs to... How about we sort this chart by frequency instead of sorting it alphabetically?

In [ ]:
df.sort_values('count', ascending=False).plot.bar(x='letter', y='count', rot=0, figsize=(10, 5));

Our Lucretian alliterative pair "winner" is *c* with 890 instances, followed closely by *a* (868) and then *p* (670).

One of the most exciting aspects of exploratory philological work is the way one experiment can lead to another completely unexpected experiment. This last graph, for example, has made me curious to know—what is that single instance of *b* alliteration in Lucretius? What does it look like?

In [ ]:
for alliterative_pair in alliterative_pairs:
    if alliterative_pair[1][0][0] == 'b':
        print(alliterative_pair)
        print('\n')
        break

Let's look a bit more closely at the surrounding lines...

In [ ]:
from string import Template
from IPython.core.display import HTML

row_template = Template('<lucr. 4.${line_number}>')
citations = [row_template.substitute(line_number=i) for i in range(542, 549)]

passage = []

for citation in citations:
    passage.append(lucretius_docrows[3][citation])

passage = "\n".join(passage)
passage_formatted = passage.replace('b', '<b>b</b>').replace('\n', '<br>')
display(HTML(passage_formatted))

It looks like the clause that contains our *b* alliterative pair is invested in some sound-related wordplay...

In [ ]:
citations = [row_template.substitute(line_number=i) for i in range(545, 547)]

b = []

for citation in citations:
    line = preprocess(lucretius_docrows[3][citation])
    line_words = line.split()
    for word in line_words:
        if 'b' in word:
            b.append(word)

print(f'There are {len(b)} words with a b in this passage: {b}')

In [ ]:
print(f'There are {passage.count("b")} instances of the letter b in this passage.')

As we read Latin literature, we inevitably if subsconsciously make a mental accounting of various literary feature. "There are a lot of *v* consonants in this sentence." "Lucretius appears to use the word patten 'v_____ v_____' more than Virgil" "I think I have seen this word pair before." etc. Again, as readers, we are good at this mental accounting in an impressionistic way. We are not as good at collecting, counting, arranging, and presenting this information, at least not without some increased level of deliberate effort. Computers can help us bridge the gap between "there are a lot of *v* consonants" and "there are five—and exactly five—*v* consonsants" in a given sentence. Our Python code can be seen then as an extension of our existing reading practice and with that extended reading practice at hand we are in an excellent position to not only **Discover** more features latent in the texts, but then to use them as the basis for arguments about the texts themselves and in comparison with other texts. It is still up to us as philologists to analyze and interpret what we find, but can do so now from a faster, more systematic, and more comprehensive position.

## Explore

### Next steps

- ***Change author***: How do rates of alliteration in Lucretius compare with, say, that of Ovid? Statius? We could try the experiment again a different author? Or how about a prose author? Does alliteration mean the same thing or work the same way in prose as it does in verse? On open (and interesting question!)
- ***Change formalization***: We used a specific Python definition of alliteration, but surely we can imagine others. How about formalizations that take into account more than two words at a time? Or establish some interest in not just the first letters of each word but of letters appear anywhere in the line? or sentence? Speaking of which, is there units of analysis that make more or less sense when searching for alliteration, i.e. lines can surely be alliterative—can sentences, paragraphs, books, entire works?

### For the future

- ***Really change the formalization*** In other experiments in *Exploratory Philology*, you will be presented with similarly structured walkthroughs of other poetic phenomena, like acrostics and telestichs, anagrams, palindromes, and more. In a certain sense, these experiments are just refactoring variations on this alliteration experiment: at the heart of these other experiments is often a shared pattern of 1. create a container/counter; 2. iterate over words, sentences, lines, paragraphs, etc.; 3. check some string feature, whether the string is manipulated or not; and 4. report results. Pick an poetic formalism other than alliteration and try to think through how you could map a **Discover** experiment onto that four-step pseudocode pattern.

## Further Reading
- Greenberg, N.A. 1980. “Aspects of Alliteration : A Statistical Study.” *Latomus* 39: 585–611.
- Hendren, G. 2012. “Woven Alliteration in the De Rerum Natura.” *CJ* 107(4): 409. doi:10.5184/classicalj.107.4.0409.
- Loch, E. 1865. *De Usu Alliterationis apud Poetas Latinos*. Typis Orphanotrophei.
- Merrill, W.A. 1892. “Alliteration in Lucretius.” *TAPA* 23.
- Munro, H.A.J., ed. 1873. T. Lucreti Cari *De Rerum Natura Libri Sex*. Cambridge: Deighton Bell and Co.
- Naekius, A.F. 1829. “De Alliteratione Sermonis Latini.” *RhM* 3: 324–418.
- Peck, T. 1884. “Alliteration in Latin.” *TAPA* 15: 58–65. doi:10.2307/2935800.